# 신경망(NEURAL NETWORKS)

- 신경망은 `torch.nn` 패키지를 사용하여 생성
- `nn.Module` 은 계층(layer)과 output 을 반환하는 `forward(input)` 메서드를 가지고 있음

## Ex) MNIST

<img src="./assets/mnist.png" alt="mnist" width="80%">

### 신경망의 일반적인 학습 과정
- 신경망을 정의
- 데이터 입력
- 입력을 신경망에서 전파(process)
- 손실(loss) 계산
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파
- 신경망의 가중치를 갱신
    - 새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3, stride=1)
        
        self.fc1 = nn.Linear(576, 120, bias=True)
        self.fc2 = nn.Linear(120, 84, bias=True)
        self.fc3 = nn.Linear(84, 10, bias=True)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        
        x = x.view(-1, 576)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    

net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params)

10
[Parameter containing:
tensor([[[[-0.1100, -0.0040, -0.2877],
          [-0.1930, -0.3166, -0.0405],
          [ 0.0206, -0.2227,  0.1524]]],


        [[[-0.2513, -0.2313, -0.2929],
          [ 0.2572,  0.1678, -0.0357],
          [ 0.0723,  0.2579, -0.0019]]],


        [[[-0.0046,  0.3114, -0.1511],
          [-0.2501, -0.2267, -0.1164],
          [-0.0439, -0.1149, -0.0141]]],


        [[[ 0.0257,  0.2321, -0.1160],
          [ 0.0998, -0.0865,  0.1618],
          [-0.1328, -0.1694,  0.2830]]],


        [[[-0.2482, -0.2911, -0.1463],
          [ 0.0220,  0.1064,  0.0396],
          [ 0.2348, -0.3127,  0.3030]]],


        [[[ 0.1639,  0.0270, -0.3050],
          [ 0.0141,  0.2314,  0.1256],
          [-0.2980, -0.1281,  0.1763]]]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2378, -0.1255,  0.0523,  0.2842,  0.1165,  0.1721], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([[[[-3.1837e-03,  1.3199e-01, -1.1380e-01],
        

### NOTE

- `torch.nn` 은 미니-배치(mini-batch)만 지원
- `nn.Conv2d()`는 `nSamples x nChannels x Height x Width` 의 4차원 Tensor를 입력받음
- 만약 하나의 샘플만 있다면, `input.unsqueeze(0)` 을 사용해서 미니배치 차원을 추가해주어야 함



In [4]:
inp = torch.randn(1, 1, 32, 32).to(device)
out = net(inp)
print(out.size())

torch.Size([1, 10])


In [5]:
# net.zero_grad()
out.backward(torch.randn((1, 10)).to(device))

# 손실 함수 (Loss Function)

- 손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져있는지 추정하는 값을 계산
- [Pytorch의 여러가지 Loss Functions](http://pytorch.org/docs/nn.html#loss-functions)

In [6]:
output = net(inp)
target = torch.randn(10).to(device)
target = target.view(1, -1)
criterion = nn.MSELoss()

# Mean-Squared Error
loss = criterion(output, target)
print(loss)

tensor(0.8171, device='cuda:0', grad_fn=<MseLossBackward>)


`loss.backward()` 를 실행할 때, 전체 그래프는 손실(loss)에 대하여 미분되며, 그래프 내의 requires_grad=True 인 모든 Tensor는 변화도(gradient)가 누적된 `.grad` Tensor를 갖게 됨

In [7]:
# 역전파를 따라가면서 찍어보기
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

# 역전파(Backprop)

- `loss.backward()` : 오차(error)를 역전파
- 이때, 이전에 계산된 gradient를 없애지 않으면 gradient가 계속 누적되므로, `zero_grad()`를 호출해주어야 함

In [8]:
net.zero_grad() # network의 모든 variable에 대한 gradient를 0으로 만듦

print('conv1.bias.grad before backprop')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backprop')
print(net.conv1.bias.grad)

# Network의 모든 parameter에 대한 gradient 출력
# for f in net.parameters():
#     print(f.grad)

conv1.bias.grad before backprop
tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')
conv1.bias.grad after backprop
tensor([ 0.0120,  0.0009, -0.0071,  0.0056, -0.0157,  0.0195], device='cuda:0')


# 가중치 갱신

- SGD(Stochastic Gradient Descent)

> ```
> 새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)
> ```

In [9]:
# 가중치 업데이트 파이썬 구현
lr = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

In [10]:
import torch.optim as optim

# optimizer 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 가중치 0으로 초기화
optimizer.zero_grad()

# Network 출력 계산
output = net(inp)

# Loss 계산
loss = criterion(output, target)

# Backprop
loss.backward()

# 가중치 업데이트
optimizer.step()

# Summary

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [12]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16 * 6 * 6)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [13]:
net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [14]:
inp = torch.randn(1, 1, 32, 32).to(device)
out = net(inp).to(device)
tgt = torch.randn(1, 10).to(device)

In [15]:
criterion = nn.MSELoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)

In [16]:
optimizer.zero_grad()
loss = criterion(out, tgt)
loss.backward()
optimizer.step()